# Ollama + PostgreSQL Vector Database 질의응답 시스템

이 노트북은 PostgreSQL + pgvector를 사용하여 문서 임베딩을 저장하고,  
벡터 유사도 검색을 통해 더 정확한 질의응답을 수행합니다.

In [ ]:
import os
import json
import numpy as np
import pandas as pd
from pathlib import Path
from typing import List, Dict, Optional, Tuple
from IPython.display import display, Markdown

# Vector Database
import psycopg2
from pgvector.psycopg2 import register_vector

# Ollama
import ollama

print("라이브러리 로딩 완료 ✅")

## 1. 데이터베이스 설정 및 연결

In [ ]:
# 데이터베이스 설정
DB_CONFIG = {
    'host': 'localhost',
    'port': 5432,
    'database': 'vector_qa',
    'user': 'postgres',
    'password': 'postgres'
}

# 임베딩 설정
EMBEDDING_MODEL = 'nomic-embed-text'  # Ollama 임베딩 모델
EMBEDDING_DIMENSION = 768  # 임베딩 차원수
CHUNK_SIZE = 1000  # 텍스트 청크 크기

print("설정 완료 ⚙️")

In [ ]:
class VectorDatabase:
    def __init__(self, config: dict):
        self.config = config
        self.conn = None
        self.embedding_model = EMBEDDING_MODEL
        self.embedding_dim = EMBEDDING_DIMENSION
    
    def connect(self) -> bool:
        """데이터베이스에 연결"""
        try:
            self.conn = psycopg2.connect(**self.config)
            self.conn.autocommit = True
            register_vector(self.conn)
            print("✅ PostgreSQL 연결 성공")
            return True
        except Exception as e:
            print(f"❌ PostgreSQL 연결 실패: {e}")
            print("\n💡 PostgreSQL 서버가 실행 중인지 확인하세요:")
            print("   brew services start postgresql@14")
            print("   또는 Docker: docker run -d -p 5432:5432 -e POSTGRES_PASSWORD=postgres postgres:14")
            return False
    
    def create_database_if_not_exists(self):
        """데이터베이스가 없으면 생성"""
        try:
            # postgres 데이터베이스에 연결하여 vector_qa 데이터베이스 생성
            temp_config = self.config.copy()
            temp_config['database'] = 'postgres'
            
            temp_conn = psycopg2.connect(**temp_config)
            temp_conn.autocommit = True
            
            with temp_conn.cursor() as cur:
                # 데이터베이스 존재 확인
                cur.execute("SELECT 1 FROM pg_database WHERE datname = %s", (self.config['database'],))
                if not cur.fetchone():
                    cur.execute(f"CREATE DATABASE {self.config['database']}")
                    print(f"✅ 데이터베이스 '{self.config['database']}' 생성 완료")
                else:
                    print(f"ℹ️  데이터베이스 '{self.config['database']}' 이미 존재")
            
            temp_conn.close()
        except Exception as e:
            print(f"❌ 데이터베이스 생성 실패: {e}")
    
    def setup_tables(self):
        """필요한 테이블 및 확장 설정"""
        if not self.conn:
            print("❌ 데이터베이스 연결이 필요합니다")
            return False
        
        try:
            with self.conn.cursor() as cur:
                # pgvector 확장 설치
                cur.execute("CREATE EXTENSION IF NOT EXISTS vector")
                print("✅ pgvector 확장 설치 완료")
                
                # 문서 테이블 생성
                cur.execute(f"""
                    CREATE TABLE IF NOT EXISTS documents (
                        id SERIAL PRIMARY KEY,
                        filename VARCHAR(255) NOT NULL,
                        content TEXT NOT NULL,
                        chunk_index INTEGER NOT NULL,
                        chunk_text TEXT NOT NULL,
                        embedding vector({self.embedding_dim}),
                        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                        metadata JSONB
                    )
                """)
                
                # 벡터 유사도 검색을 위한 인덱스 생성
                cur.execute("""
                    CREATE INDEX IF NOT EXISTS documents_embedding_idx 
                    ON documents USING ivfflat (embedding vector_cosine_ops) 
                    WITH (lists = 100)
                """)
                
                print("✅ 테이블 및 인덱스 설정 완료")
                return True
                
        except Exception as e:
            print(f"❌ 테이블 설정 실패: {e}")
            return False
    
    def close(self):
        """데이터베이스 연결 종료"""
        if self.conn:
            self.conn.close()
            print("🔒 데이터베이스 연결 종료")

# 벡터 데이터베이스 초기화
vector_db = VectorDatabase(DB_CONFIG)
print("VectorDatabase 클래스 정의 완료 🗄️")

## 2. 데이터베이스 초기화 및 연결 테스트

In [ ]:
# 데이터베이스 생성 (필요한 경우)
vector_db.create_database_if_not_exists()

# 데이터베이스 연결
if vector_db.connect():
    # 테이블 설정
    vector_db.setup_tables()
else:
    print("⚠️  데이터베이스 연결에 실패했습니다. PostgreSQL 설정을 확인해주세요.")

## 3. 임베딩 생성 및 문서 처리 함수

In [ ]:
def generate_embedding(text: str, model: str = EMBEDDING_MODEL) -> List[float]:
    """
    Ollama를 사용하여 텍스트 임베딩 생성
    
    Args:
        text (str): 임베딩할 텍스트
        model (str): 사용할 임베딩 모델
    
    Returns:
        List[float]: 임베딩 벡터
    """
    try:
        response = ollama.embeddings(model=model, prompt=text)
        return response['embedding']
    except Exception as e:
        print(f"❌ 임베딩 생성 실패: {e}")
        print(f"💡 '{model}' 모델이 설치되어 있는지 확인하세요: ollama pull {model}")
        return None

def chunk_text(text: str, chunk_size: int = CHUNK_SIZE, overlap: int = 100) -> List[str]:
    """
    긴 텍스트를 청크로 분할
    
    Args:
        text (str): 분할할 텍스트
        chunk_size (int): 청크 크기
        overlap (int): 청크 간 겹치는 부분
    
    Returns:
        List[str]: 텍스트 청크 리스트
    """
    if len(text) <= chunk_size:
        return [text]
    
    chunks = []
    start = 0
    
    while start < len(text):
        end = start + chunk_size
        
        if end >= len(text):
            chunks.append(text[start:])
            break
        
        # 단어 경계에서 자르기
        while end > start and text[end] not in ' \n\t.!?;':
            end -= 1
        
        if end == start:
            end = start + chunk_size
        
        chunks.append(text[start:end])
        start = end - overlap
    
    return chunks

def load_and_embed_documents(doc_folder: str = 'doc') -> bool:
    """
    문서를 로딩하고 임베딩을 생성하여 데이터베이스에 저장
    
    Args:
        doc_folder (str): 문서 폴더 경로
    
    Returns:
        bool: 성공 여부
    """
    if not vector_db.conn:
        print("❌ 데이터베이스 연결이 필요합니다")
        return False
    
    doc_path = Path(doc_folder)
    if not doc_path.exists():
        print(f"❌ '{doc_folder}' 폴더가 존재하지 않습니다")
        return False
    
    md_files = list(doc_path.glob('*.md'))
    if not md_files:
        print(f"❌ '{doc_folder}' 폴더에 마크다운 파일이 없습니다")
        return False
    
    print(f"📚 {len(md_files)}개 문서 처리 시작...")
    
    try:
        with vector_db.conn.cursor() as cur:
            # 기존 데이터 삭제 (선택사항)
            # cur.execute("DELETE FROM documents")
            
            total_chunks = 0
            
            for md_file in md_files:
                print(f"\n📄 처리 중: {md_file.name}")
                
                try:
                    with open(md_file, 'r', encoding='utf-8') as f:
                        content = f.read()
                    
                    # 텍스트 청킹
                    chunks = chunk_text(content)
                    print(f"  📝 {len(chunks)}개 청크로 분할")
                    
                    for i, chunk in enumerate(chunks):
                        if len(chunk.strip()) < 50:  # 너무 짧은 청크는 스킵
                            continue
                        
                        # 임베딩 생성
                        embedding = generate_embedding(chunk)
                        if embedding is None:
                            continue
                        
                        # 데이터베이스에 저장
                        metadata = {
                            'file_size': len(content),
                            'chunk_size': len(chunk)
                        }
                        
                        cur.execute("""
                            INSERT INTO documents 
                            (filename, content, chunk_index, chunk_text, embedding, metadata)
                            VALUES (%s, %s, %s, %s, %s, %s)
                        """, (
                            md_file.name,
                            content,
                            i,
                            chunk,
                            embedding,
                            json.dumps(metadata)
                        ))
                        
                        total_chunks += 1
                        
                        if (i + 1) % 5 == 0:
                            print(f"    ⚡ {i + 1}개 청크 처리 완료")
                
                except Exception as e:
                    print(f"  ❌ {md_file.name} 처리 실패: {e}")
                    continue
            
            print(f"\n✅ 총 {total_chunks}개 청크를 데이터베이스에 저장했습니다")
            return True
            
    except Exception as e:
        print(f"❌ 문서 임베딩 처리 실패: {e}")
        return False

print("문서 처리 함수 정의 완료 📚")

## 4. 문서 로딩 및 임베딩 생성

In [ ]:
# 임베딩 모델 다운로드 확인
try:
    test_embedding = generate_embedding("테스트 텍스트")
    if test_embedding:
        print(f"✅ 임베딩 모델 '{EMBEDDING_MODEL}' 사용 가능")
        print(f"📏 임베딩 차원: {len(test_embedding)}")
        
        # 임베딩 차원 업데이트 (실제 모델의 차원과 맞춤)
        if len(test_embedding) != EMBEDDING_DIMENSION:
            EMBEDDING_DIMENSION = len(test_embedding)
            vector_db.embedding_dim = EMBEDDING_DIMENSION
            print(f"🔧 임베딩 차원을 {EMBEDDING_DIMENSION}으로 업데이트")
    else:
        print(f"❌ 임베딩 모델을 다운로드하세요: ollama pull {EMBEDDING_MODEL}")
except Exception as e:
    print(f"❌ 임베딩 테스트 실패: {e}")

In [ ]:
# 문서 로딩 및 임베딩 생성 실행
if vector_db.conn:
    success = load_and_embed_documents('doc')
    if success:
        # 저장된 문서 통계 확인
        with vector_db.conn.cursor() as cur:
            cur.execute("SELECT COUNT(*) FROM documents")
            total_chunks = cur.fetchone()[0]
            
            cur.execute("SELECT COUNT(DISTINCT filename) FROM documents")
            total_files = cur.fetchone()[0]
            
            print(f"\n📊 데이터베이스 현황:")
            print(f"  📁 파일 수: {total_files}개")
            print(f"  📄 청크 수: {total_chunks}개")
else:
    print("⚠️  데이터베이스 연결이 필요합니다")

## 5. 벡터 유사도 검색 및 질의응답 함수

In [ ]:
def search_similar_chunks(query: str, limit: int = 5, similarity_threshold: float = 0.7) -> List[Dict]:
    """
    벡터 유사도를 사용하여 관련 문서 청크 검색
    
    Args:
        query (str): 검색 쿼리
        limit (int): 반환할 최대 결과 수
        similarity_threshold (float): 유사도 임계값 (0~1)
    
    Returns:
        List[Dict]: 유사한 문서 청크 리스트
    """
    if not vector_db.conn:
        print("❌ 데이터베이스 연결이 필요합니다")
        return []
    
    # 쿼리 임베딩 생성
    query_embedding = generate_embedding(query)
    if not query_embedding:
        print("❌ 쿼리 임베딩 생성 실패")
        return []
    
    try:
        with vector_db.conn.cursor() as cur:
            # 코사인 유사도 검색
            cur.execute("""
                SELECT 
                    id,
                    filename,
                    chunk_index,
                    chunk_text,
                    metadata,
                    1 - (embedding <=> %s) AS similarity
                FROM documents
                WHERE 1 - (embedding <=> %s) > %s
                ORDER BY embedding <=> %s
                LIMIT %s
            """, (query_embedding, query_embedding, similarity_threshold, query_embedding, limit))
            
            results = []
            for row in cur.fetchall():
                results.append({
                    'id': row[0],
                    'filename': row[1],
                    'chunk_index': row[2],
                    'chunk_text': row[3],
                    'metadata': json.loads(row[4]) if row[4] else {},
                    'similarity': float(row[5])
                })
            
            return results
            
    except Exception as e:
        print(f"❌ 벡터 검색 실패: {e}")
        return []

def vector_qa(question: str, model_name: str = 'llama3.1', max_context_length: int = 4000) -> str:
    """
    벡터 검색 기반 질의응답
    
    Args:
        question (str): 질문
        model_name (str): 사용할 Ollama 모델
        max_context_length (int): 최대 컨텍스트 길이
    
    Returns:
        str: 답변
    """
    print(f"🔍 질문 분석 중: {question}")
    
    # 관련 문서 청크 검색
    similar_chunks = search_similar_chunks(question, limit=10, similarity_threshold=0.3)
    
    if not similar_chunks:
        return "❌ 관련된 문서를 찾을 수 없습니다. 다른 질문을 시도해보세요."
    
    print(f"📚 {len(similar_chunks)}개 관련 문서 청크 발견")
    
    # 컨텍스트 구성 (길이 제한)
    context_parts = []
    current_length = 0
    
    for chunk in similar_chunks:
        chunk_text = f"=== {chunk['filename']} (유사도: {chunk['similarity']:.3f}) ===\n{chunk['chunk_text']}\n"
        
        if current_length + len(chunk_text) > max_context_length:
            break
        
        context_parts.append(chunk_text)
        current_length += len(chunk_text)
    
    context = "\n".join(context_parts)
    
    # 프롬프트 구성
    prompt = f"""다음 문서들을 기반으로 질문에 정확하고 자세하게 답변해주세요:

관련 문서:
{context}

질문: {question}

답변 규칙:
1. 문서의 내용만을 기반으로 답변하세요
2. 문서에 없는 내용은 추측하지 마세요
3. 가능하면 구체적인 예시나 코드를 포함하세요
4. 답변의 근거가 되는 문서명을 언급하세요

답변:"""
    
    try:
        print(f"🤖 {model_name} 모델로 답변 생성 중...")
        
        response = ollama.chat(model=model_name, messages=[
            {
                'role': 'user',
                'content': prompt
            }
        ])
        
        answer = response['message']['content']
        
        # 참고 문서 정보 추가
        referenced_files = set(chunk['filename'] for chunk in similar_chunks[:5])
        references = f"\n\n📋 **참고 문서:** {', '.join(referenced_files)}"
        
        return answer + references
        
    except Exception as e:
        return f"❌ 답변 생성 실패: {e}"

def show_similar_chunks(question: str, limit: int = 5):
    """
    질문과 유사한 문서 청크들을 표시
    
    Args:
        question (str): 검색 질문
        limit (int): 표시할 결과 수
    """
    chunks = search_similar_chunks(question, limit=limit, similarity_threshold=0.2)
    
    if not chunks:
        print("❌ 유사한 문서를 찾을 수 없습니다")
        return
    
    print(f"\n🔍 '{question}'과 유사한 문서 청크들:")
    print("=" * 80)
    
    for i, chunk in enumerate(chunks, 1):
        print(f"\n{i}. 📄 {chunk['filename']} (청크 {chunk['chunk_index']})")
        print(f"   🎯 유사도: {chunk['similarity']:.3f}")
        print(f"   📝 내용: {chunk['chunk_text'][:200]}{'...' if len(chunk['chunk_text']) > 200 else ''}")
        print("-" * 80)

print("벡터 검색 및 QA 함수 정의 완료 🔍")

## 6. 벡터 검색 테스트

In [ ]:
# 벡터 검색 테스트
if vector_db.conn:
    test_query = "Network API 사용법"
    print(f"🧪 테스트 쿼리: '{test_query}'")
    
    # 유사 문서 청크 표시
    show_similar_chunks(test_query, limit=3)
else:
    print("⚠️  데이터베이스 연결이 필요합니다")

## 7. 벡터 기반 질의응답 예시

In [ ]:
# 예시 질의응답
sample_questions = [
    "파일 업로드는 어떻게 하나요?",
    "카메라 촬영 기능을 사용하는 방법은?",
    "JWT 토큰 관리는 어떻게 해야 하나요?"
]

if vector_db.conn:
    print("🤖 벡터 기반 질의응답 예시")
    print("=" * 60)
    
    for i, question in enumerate(sample_questions, 1):
        print(f"\n{i}. ❓ 질문: {question}")
        print("-" * 50)
        
        answer = vector_qa(question)
        print(f"💬 답변:\n{answer}")
        print("=" * 60)
        
        if i < len(sample_questions):
            input("\n⏳ 다음 질문으로 넘어가려면 Enter를 누르세요...")
else:
    print("⚠️  데이터베이스 연결이 필요합니다")

## 8. 대화형 벡터 질의응답

In [ ]:
def interactive_vector_qa():
    """
    대화형 벡터 기반 질의응답 인터페이스
    """
    if not vector_db.conn:
        print("❌ 데이터베이스 연결이 필요합니다")
        return
    
    print("=" * 70)
    print("🚀 벡터 데이터베이스 기반 질의응답 시스템")
    print("=" * 70)
    print("특징:")
    print("  🔍 벡터 유사도 검색으로 정확한 문서 검색")
    print("  📚 PostgreSQL + pgvector 활용")
    print("  🎯 문서 청킹 및 임베딩 기반 검색")
    print()
    print("사용법:")
    print("  - 질문을 입력하면 관련 문서를 검색하여 답변합니다")
    print("  - 'quit' 또는 'exit'로 종료")
    print("  - '!search [질문]'으로 유사 문서만 검색")
    print("  - '!stats'로 데이터베이스 통계 확인")
    print("-" * 70)
    
    while True:
        question = input("\n🤖 질문을 입력하세요: ").strip()
        
        if question.lower() in ['quit', 'exit', '종료']:
            print("👋 벡터 질의응답을 종료합니다.")
            break
        
        if question.startswith('!search '):
            search_query = question[8:].strip()
            show_similar_chunks(search_query, limit=5)
            continue
        
        if question == '!stats':
            try:
                with vector_db.conn.cursor() as cur:
                    cur.execute("SELECT COUNT(*) FROM documents")
                    total_chunks = cur.fetchone()[0]
                    
                    cur.execute("SELECT COUNT(DISTINCT filename) FROM documents")
                    total_files = cur.fetchone()[0]
                    
                    cur.execute("SELECT filename, COUNT(*) FROM documents GROUP BY filename")
                    file_stats = cur.fetchall()
                    
                    print(f"\n📊 데이터베이스 통계:")
                    print(f"  📁 총 파일 수: {total_files}개")
                    print(f"  📄 총 청크 수: {total_chunks}개")
                    print(f"  📋 파일별 청크 수:")
                    for filename, count in file_stats:
                        print(f"    - {filename}: {count}개")
            except Exception as e:
                print(f"❌ 통계 조회 실패: {e}")
            continue
        
        if not question:
            print("질문을 입력해주세요.")
            continue
        
        print("\n" + "="*50)
        answer = vector_qa(question)
        print("\n💬 답변:")
        print(answer)
        print("="*50)

# 대화형 인터페이스 시작 (필요시 실행)
# interactive_vector_qa()

print("대화형 벡터 QA 함수 정의 완료 💬")

## 9. 커스텀 벡터 질의

In [ ]:
# 직접 질문 입력하여 테스트
custom_question = "bizMOB에서 데이터베이스 연결하는 방법을 알려주세요"

if custom_question.strip() and vector_db.conn:
    print(f"❓ 사용자 질문: {custom_question}")
    print("="*60)
    
    # 벡터 기반 답변 생성
    answer = vector_qa(custom_question)
    print(f"\n💬 벡터 검색 기반 답변:")
    display(Markdown(answer))
    
    print("\n🔍 관련 문서 청크 분석:")
    show_similar_chunks(custom_question, limit=3)
elif not custom_question.strip():
    print("위의 custom_question 변수에 질문을 입력하고 셀을 실행하세요.")
else:
    print("⚠️  데이터베이스 연결이 필요합니다")

## 10. 성능 비교 및 분석 도구

In [ ]:
def compare_search_methods(question: str):
    """
    벡터 검색과 기존 텍스트 검색 비교
    
    Args:
        question (str): 비교할 질문
    """
    if not vector_db.conn:
        print("❌ 데이터베이스 연결이 필요합니다")
        return
    
    print(f"⚔️  검색 방법 비교: '{question}'")
    print("=" * 80)
    
    # 1. 벡터 검색
    print("\n🔍 1. 벡터 유사도 검색 결과:")
    print("-" * 40)
    vector_results = search_similar_chunks(question, limit=3, similarity_threshold=0.2)
    
    if vector_results:
        for i, result in enumerate(vector_results, 1):
            print(f"{i}. {result['filename']} (유사도: {result['similarity']:.3f})")
            print(f"   📝 {result['chunk_text'][:100]}...")
    else:
        print("  검색 결과 없음")
    
    # 2. 키워드 검색 (PostgreSQL 전문검색)
    print("\n🔤 2. 키워드 검색 결과:")
    print("-" * 40)
    
    try:
        with vector_db.conn.cursor() as cur:
            # 간단한 키워드 검색
            search_terms = question.split()
            keyword_query = ' | '.join(search_terms[:3])  # 처음 3개 단어만 사용
            
            cur.execute("""
                SELECT filename, chunk_text, 
                       ts_rank(to_tsvector('english', chunk_text), 
                               plainto_tsquery('english', %s)) as rank
                FROM documents
                WHERE to_tsvector('english', chunk_text) @@ plainto_tsquery('english', %s)
                ORDER BY rank DESC
                LIMIT 3
            """, (question, question))
            
            keyword_results = cur.fetchall()
            
            if keyword_results:
                for i, result in enumerate(keyword_results, 1):
                    print(f"{i}. {result[0]} (랭크: {result[2]:.4f})")
                    print(f"   📝 {result[1][:100]}...")
            else:
                print("  검색 결과 없음")
                
    except Exception as e:
        print(f"  키워드 검색 오류: {e}")
    
    print("\n" + "=" * 80)

def analyze_embeddings():
    """
    저장된 임베딩 데이터 분석
    """
    if not vector_db.conn:
        print("❌ 데이터베이스 연결이 필요합니다")
        return
    
    try:
        with vector_db.conn.cursor() as cur:
            # 기본 통계
            cur.execute("""
                SELECT 
                    filename,
                    COUNT(*) as chunk_count,
                    AVG(LENGTH(chunk_text)) as avg_chunk_length,
                    MAX(LENGTH(chunk_text)) as max_chunk_length,
                    MIN(LENGTH(chunk_text)) as min_chunk_length
                FROM documents
                GROUP BY filename
                ORDER BY chunk_count DESC
            """)
            
            results = cur.fetchall()
            
            print("📊 임베딩 데이터 분석")
            print("=" * 80)
            
            df_stats = pd.DataFrame(results, columns=[
                '파일명', '청크수', '평균길이', '최대길이', '최소길이'
            ])
            
            # 수치 컬럼 반올림
            df_stats['평균길이'] = df_stats['평균길이'].round(0).astype(int)
            
            display(df_stats)
            
            # 전체 요약
            total_chunks = df_stats['청크수'].sum()
            avg_chunk_size = df_stats['평균길이'].mean()
            
            print(f"\n📋 전체 요약:")
            print(f"  • 총 청크 수: {total_chunks:,}개")
            print(f"  • 평균 청크 크기: {avg_chunk_size:.0f}자")
            print(f"  • 임베딩 차원: {EMBEDDING_DIMENSION}")
            print(f"  • 사용 모델: {EMBEDDING_MODEL}")
            
    except Exception as e:
        print(f"❌ 분석 실패: {e}")

print("성능 분석 도구 정의 완료 📈")

In [ ]:
# 성능 분석 실행
if vector_db.conn:
    print("📈 임베딩 데이터 분석")
    analyze_embeddings()
    
    print("\n" + "="*80)
    print("⚔️  검색 방법 비교 테스트")
    compare_search_methods("파일 업로드 API 사용법")
else:
    print("⚠️  데이터베이스 연결이 필요합니다")

## 11. 정리 및 리소스 해제

In [ ]:
# 데이터베이스 연결 종료 (필요시)
# vector_db.close()

print("""
🎉 PostgreSQL + pgvector 기반 질의응답 시스템 완료!

✅ 주요 기능:
  • 문서 자동 청킹 및 임베딩 생성
  • PostgreSQL + pgvector 벡터 저장
  • 코사인 유사도 기반 검색
  • Ollama 모델과 연동된 질의응답
  • 성능 분석 및 비교 도구

🔧 사용 가능한 함수들:
  • vector_qa(question) - 벡터 기반 질의응답
  • search_similar_chunks(query) - 유사 문서 검색
  • show_similar_chunks(question) - 검색 결과 표시
  • interactive_vector_qa() - 대화형 인터페이스
  • compare_search_methods(question) - 검색 방법 비교
  • analyze_embeddings() - 임베딩 데이터 분석

💡 장점:
  • 더 정확한 의미 기반 검색
  • 대용량 문서 처리 가능
  • 확장 가능한 벡터 데이터베이스
  • 실시간 검색 성능
""")